In [1]:
from dotenv import load_dotenv
# from pytesseract import image_to_string
from PIL import Image
from io import BytesIO
# import pypdfium2 as pdfium
import streamlit as st
import multiprocessing
# from tempfile import NamedTemporaryFile
import pandas as pd
import json
import requests
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
import os
# from tempfile import NamedTemporaryFile
from jsonformer.format import highlight_values
# from jsonformer.main import Jsonformer
from langchain_experimental.llms import JsonFormer
from transformers import TextStreamer, pipeline

from langchain import PromptTemplate
from langchain.chains import LLMChain
import time
load_dotenv()

True

In [2]:
nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)


model_id = 'mychen76/mistral7b_ocr_to_json_v1'
cache_dir = '/u01/tmp/mistral_7b_json'

# model_id = 'mistralai/Mistral-7B-v0.1'
# cache_dir = '/home/vinbig/Documents/PA_Modeling/Prompt/mistral_tmp'

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=nf4_config,
    # device_map="auto",
    cache_dir=cache_dir,
    
)
tokenizer = AutoTokenizer.from_pretrained(model_id, cache_dir=cache_dir)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
template = """
You are an AI Assistant in the medical field. Your goal is to provide the Human with information extracted from the Human"s prescription. Think step by step and never skip any step.
Please try to extract all data points. Do not add or omit any information. If you don"t know, just answer "don"t know" and do not include information that is not in the document in your answer.
{data_points}
    
EXAMPLES
----
Human: BỆNH VIỆN VIETT ĐỨC Số toa: 71 Nhà thuốc Bệnh viện Số điện thoại: 02435766328 Năm sinh: 1963 15A-Phương Mai-Đống Đa-Hà Nội PHIÊU THU TIỀN Xuất từ: Quầy Thuốc 1 In: Quầy Thuốc Lần in: 1 Giờ in: 08:15:54 Họ tên bệnh nhân: LÊ NGỌC LAN Mã bệnh nhân: 0029212798 Bác sĩ khám bệnh: Ths.BS TRỊNH MINH TRANG TT Tên mặt hàng ĐVT SL Đơn giá Thành tiền Spulit 100mg Viên 60 17.655 1.059.300 2 Ziaja Med Anti-imperfections Formula Cleansing Body Gel (Gel tắm ngừa khuẩn) 400ml Chai 1 499.460 499.460 3 Notis Antidanruff Shampoo 125ml Chai 2 248.600 497.200 4 Amisea 167mg Viên 30 6.420 192.600 5 Cafunten 10g Tuýp 4 6.527 26.108 Tổng khoản: 5 Tổng tiền: 2.274.668 Bằng chữ: Hai triệu hai trăm bảy mượi bốn nghìn sáu trăm sáu mươi tám đồng. Ngày 26 tháng 04 năm 2022 Người thu tiền Người nhận thuốc ngay trong ngày Lưu Trường hợp khách hàng có đơn tài chính đề nghị lấy (Quá ngày Bệnh viện không với nhân viên (Ký, họ tên) (Ký, họ tên) nhà thuốc để được hướng dẫn) Trân trọng cảm ơn Quý khách đã mua thuốc tại Bệnh viện. NGUYỄN HÀ MY LÊ NGỌC LAN
AI ASSISTANT: {{"current_institute": "BỆNH VIỆN VIỆT ĐỨC", "name": "LÊ NGỌC LAN", "gender": "", "birth": "1963", "age": "", "address": "", "tel_customer": "", "id_bhyt": "", "diagnosis": "", "date_in": "Ngày 26 tháng 04 năm 2022", "doctor_name": "Ths.BS TRỊNH MINH TRANG"}} END

Human: ! Mã BN:2101002494 Số: 211002020 Mã hồ sơ: 2110150077 Kho:110-Kho Ngoại trú ĐƠN THUỐC BẢO HIỂM KKB.43333 Giới tính: Nam Đối tượng: Số thẻ BHYT: CK2383820079366 Địa chỉ: Lê Hoàn 2, Điện Biên, Thành phố Thanh Hóa, Tỉnh Thanh Hóa, Việt Nam Chẩn đoán: E11-Bệnh đái tháo đường không phụ thuộc insuline/ E78-1 Rối loạn chuyển Phòng: Phòng khám 326 Họ tên: LÊ HỒNG KHANH hóa lipoprotein và tình trạng tăng lipid máu khác / Bảo Hiểm Nơi ĐK KCB BĐ: 38280 Ngày sinh: 18/06/1956 Tuổi: 65 SĐT: 0912 660 254 STT Tên thuốc- Cách dùng ĐVT Số lượng BETICAPO 750 SR-750mg (Metformin) Ngày uống 1 viên sau ăn chiều. Viên 60 2 Gliclada 60mg modified- release tablets (Gliclazid) 3 Ngày uống 2 viên trước ăn sáng 30 phút TV. Fenofibrat- 200mg (Fenofibrat) Viên 120 Uống tối 1 viên ngay sau ăn Viên 60 Cộng khoản: 3 loại Lưu Khám lại khi thấy bất thường và khi hết thuốc. Kế toán Thủ kho Người bệnh Ngày 15 tháng 10 năm 2021 Bác sĩ khám (Ký và ghi rõ họ, tên) (Ký và ghi rõ họ, tên) Khih Lê Văn Chinh ISOFH-Người in: Lê Văn Chinh, ngày in: 15/10/2021 08:24
AI ASSISTANT: {{"current_institute": "", "name": "LÊ HỒNG KHANH",  "gender": "Nam", "birth" : "18/06/1956", "age": "65", "address": "Lê Hoàn 2, Điện Biên, Thành phố Thanh Hóa, Tỉnh Thanh Hóa, Việt Nam", "tel_customer": "0912 660 254", "id_bhyt": "CK2383820079366", "diagnosis": "E11 - Bệnh đái tháo đường không phụ thuộc insuline / E78 - Rối loạn chuyển hóa lipoprotein và tình trạng tăng lipid máu khác", "date_in": "Ngày 15 tháng 10 năm 2021", "doctor_name": "Lê Văn Chinh"}} END

------    
Human: {content}
AI ASSISTANT:
""".strip()

data_points = """{
    "current_institute": "name of the hospital or clinic issuing the prescription",
    "name": "patient full name",
    "gender": "patient gender",
    "birth": "date of birth",
    "age": "patient age",
    "address": "patient address",
    "tel_customer": "patient phone number",
    "id_bhyt": "health insurance card number",
    "diagnosis": "diagnosis",
    "date_in": "issued date",
    "doctor_name": "doctor full name",
}"""

json_schema = {
    "type": "object",
    "properties": {
        "current_institute": {"type": "string"},
        "name": {"type": "string"},
        "gender": {"type": "string"},
        "birth": {"type": "string"},
        "age": {"type": "string"},
        "address": {"type": "string"},
        "tel_customer": {"type": "string"},
        "id_bhyt": {"type": "string"},
        "diagnosis": {"type": "string"},
        "drugs": {
            "type": "array",
            "drugs": {
                "type": "object",
                "properties": {
                    "drug_name": {"type": "string"},
                    "drug_dose": {"type": "string"},
                    "drug_quantity": {"type": "string"}
                }
            }
        },
        "date_in": {"type": "string"},
        "doctor_name": {"type": "string"},                
    }
}

 
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,
    temperature=0,
    top_k=50,
    top_p=0.95,
    repetition_penalty=1.15,
    # streamer=streamer,
    stop_sequence = ['END\n', '\n']
    )

builder = JsonFormer(json_schema=json_schema, pipeline=text_pipeline, batch_size=4)


/home/vinbig/miniconda3/envs/prompt/lib/python3.9/site-packages/transformers/pipelines/text_generation.py:148: UserWarning: Stopping on a multiple token sequence is not yet supported on transformers. The first token of the stop sequence will be used as the stop sequence string in the interim.
  warnings.warn(


In [4]:
from torch.utils.data import Dataset
import os 
import ast
import json
class CustomDataset(Dataset):
    def __init__(self, text_folder,template = template, data_points=data_points):
        self.text_files = [os.path.join(text_folder, file) for file in os.listdir(text_folder)[:50]]
        self.text_files.sort()
        self.template=template
        self.data_points=data_points

    def __len__(self):
        return len(self.text_files)
    
    def __getitem__(self, index):
        with open(self.text_files[index],'r', encoding='utf-8') as file:
            text_content = file.read()
            text = template.format(content=text_content, data_points=data_points)
        return text
    @property
    def num_rows(self):
        return len(self)
    
data = CustomDataset("/home/vinbig/Documents/PA_Modeling/Prompt/private_test_Pharma/ocr_text")

In [5]:
type(data)

__main__.CustomDataset

In [7]:
from transformers import pipeline
from torch.utils.data import Dataset
from tqdm.auto import tqdm

import time
# for batch_size in [1, 4]:
#     str = time.time()
#     print("-" * 30)
#     print(f"Streaming batch_size={batch_size}")
str = time.time()
for out in tqdm(builder.generate(data), total=len(data)):
    print(out)
print("TG:", time.time()-str)

ValueError: Argument 'prompts' is expected to be of type List[str], received argument of type <class '__main__.CustomDataset'>.